In [1]:
import tensorflow as tf
import os
import numpy as np
import pickle

CIFAR_DIR = "cifar-10-batches-py"

AttributeError: module 'tensorflow.python.training.checkpointable' has no attribute 'CheckpointableBase'

In [2]:
def load_data(fileName):
    """从文件中读取数据"""
    with open(fileName, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']
    
    
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        # 将数据缩放到[-1, 1]
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        
        self._indicator = 0  # 当前遍历到的位置
        if self._need_shuffle:
            self._shuffle_data()
        
        
    """打乱训练数据集"""
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    """返回batch_size个样本"""
    def next_batch (self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch_size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
    
"""加载数据"""  
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]


train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)





(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

In [4]:
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32 * 32
x_image = tf.transpose(x_image,  perm=[0, 2, 3, 1])

conv1 = tf.layers.conv2d(x_image, 
                         32, # 卷积核输出的通道数
                         (3, 3), #卷积核的大小
                         padding='same', #使用padding 使得图片输出大小相等
                         activation=tf.nn.relu, 
                         name='conv1')

# 16 * 16
pooling1 = tf.layers.max_pooling2d(conv1, 
                                  (2, 2), # 核大小
                                  (2, 2), # 步长
                                  name='pool1')

conv2 = tf.layers.conv2d(pooling1, 
                         32, # 卷积核输出的通道数
                         (3, 3), #卷积核的大小
                         padding='same', #使用padding 使得图片输出大小相等
                         activation=tf.nn.relu, 
                         name='conv2')

# 8 * 8
pooling2 = tf.layers.max_pooling2d(conv2, 
                                  (2, 2), # 核大小
                                  (2, 2), # 步长
                                  name='pool2')

conv3 = tf.layers.conv2d(pooling2, 
                         32, # 卷积核输出的通道数
                         (3, 3), #卷积核的大小
                         padding='same', #使用padding 使得图片输出大小相等
                         activation=tf.nn.relu, 
                         name='conv3')

#4 * 4
pooling3 = tf.layers.max_pooling2d(conv3, 
                                  (2, 2), # 核大小
                                  (2, 2), # 步长
                                  name='pool3')

#[None, 4 * 4 * 32]
flatten = tf.layers.flatten(pooling3)

y_ = tf.layers.dense(flatten, 10)

"""交叉熵损失函数"""
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_


#indices
predict = tf.arg_max(y_, 1)
corrent_preicttion = tf.equal(predict, y)
#准确率
accuracy = tf.reduce_mean(tf.cast(corrent_preicttion, tf.float64))


#梯度下降
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use `argmax` instead


In [6]:
# 初始化函数
init = tf.global_variables_initializer() 
batch_size = 20
batch_steps = 100000
test_steps = 100
#实例化session
with tf.Session() as sess:
    sess.run(init)
    for i in range(batch_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss, accuracy, train_op], feed_dict={x: batch_data, y: batch_labels})
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' % (i+1, loss_val, acc_val))
#         每5000次进行一次测试
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict = {x: test_batch_data, y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' %(i+1, test_acc))

[Train] Step: 500, loss: 1.48490, acc: 0.55000
[Train] Step: 1000, loss: 1.57237, acc: 0.35000
[Train] Step: 1500, loss: 1.03279, acc: 0.70000
[Train] Step: 2000, loss: 1.24921, acc: 0.40000
[Train] Step: 2500, loss: 0.74510, acc: 0.70000
[Train] Step: 3000, loss: 1.03426, acc: 0.65000
[Train] Step: 3500, loss: 1.13217, acc: 0.65000
[Train] Step: 4000, loss: 1.04440, acc: 0.70000
[Train] Step: 4500, loss: 1.21924, acc: 0.60000
[Train] Step: 5000, loss: 0.64973, acc: 0.75000
(10000, 3072)
(10000,)
[Test] Step: 5000, acc: 0.64400
[Train] Step: 5500, loss: 0.64130, acc: 0.80000
[Train] Step: 6000, loss: 0.64789, acc: 0.80000
[Train] Step: 6500, loss: 0.60592, acc: 0.75000
[Train] Step: 7000, loss: 0.99462, acc: 0.60000
[Train] Step: 7500, loss: 1.15653, acc: 0.55000
[Train] Step: 8000, loss: 0.59486, acc: 0.85000
[Train] Step: 8500, loss: 0.40947, acc: 0.85000
[Train] Step: 9000, loss: 0.57830, acc: 0.85000
[Train] Step: 9500, loss: 1.28861, acc: 0.55000
[Train] Step: 10000, loss: 0.97050

KeyboardInterrupt: 